In [3]:
import random as r

from engine import Value
from basic_modules import *
from basic_ops import *
from gpt import *

In [5]:
batch_size = 2
vocab_len = 10
model_dim = 4
seq_len = 5
num_heads = 2
head_dim = 2